In [ ]:
# Main requirements 

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BEG FUNTION DEF

In [ ]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img =cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img = grayscale(img)     # CONVERT TO GRAYSCALE
    img = equalize(img)      # STANDARDIZE THE LIGHTING IN AN IMAGE
    img = img/255            # TO NORMALIZE VALUES BETWEEN 0 AND 1 INSTEAD OF 0 TO 255
    return img

In [ ]:
def myModel():
    # Hyperparameter selection                                                                                                                                                                                                                   
    # Filters of the CNN                                                                                                                                                                                                         
    no_Of_Filters=60                                                                                                                                                                                                             
    # Shape of the filters used in the CNN                                                                                                                                                                                       
    size_of_Filter=(5,5)                                                                                                                                                                                                         
    size_of_Filter2=(3,3)                                                                                                                                                                                                        
    # Tekes batches of 2x2 pixels and avg the                                                                                                                                                                                    
    size_of_pool=(2,2)                                                                                                                                                                                                           
    # Nodes of the neural classifier                                                                                                                                                                                             
    no_Of_Nodes = 500                                                                                                                                                                                                            
                                                                                                                                                                                                                                 
                                                                                                                                                                                                                                 
    model= tf.keras.Sequential()                                                                                                                                                                                                 
    # Two convolutional layers                                                                                                                                                                                                   
    model.add((tf.keras.layers.Conv2D(no_Of_Filters,size_of_Filter,input_shape=(imageDimesions[0],imageDimesions[1],1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE 
    model.add((tf.keras.layers.Conv2D(no_Of_Filters, size_of_Filter, activation='relu')))                                                                                                                                        
    # Pooling to scale down the data                                                                                                                                                                                             
    model.add(tf.keras.layers.MaxPooling2D(pool_size=size_of_pool)) # DOES NOT EFFECT THE DEPTH/NO OF FILTERS                                                                                                                    
                                                                                                                                                                                                                                 
    # Repeat the same process                                                                                                                                                                                                    
    model.add((tf.keras.layers.Conv2D(no_Of_Filters//2, size_of_Filter2,activation='relu')))                                                                                                                                     
    model.add((tf.keras.layers.Conv2D(no_Of_Filters//2, size_of_Filter2, activation='relu')))                                                                                                                                  
    model.add(tf.keras.layers.MaxPooling2D(pool_size=size_of_pool))                                                                                                                                                              
    # Add a dropout -> we randomly shut down some neurons in order to prevent overfitting                                                                                                                                        
    model.add(tf.keras.layers.Dropout(0.5))                                                                                                                                                                                      
                                                                                                                                                                                                                                 
    # End of the feature extraction, now we need use the classifier                                                                                                                                                              
                                                                                                                                                                                                                                 
    # First set of layers                                                                                                                                                                                                        
    model.add(tf.keras.layers.Flatten())                                                                                                                                                                                         
    # One layer of 500 nodes densly contected with the features (output of CNN)                                                                                                                                                  
    model.add(tf.keras.layers.Dense(no_Of_Nodes,activation='relu'))                                                                                                                                                              
                                                                                                                                                                                                                                 
    # Add a dropout -> we randomly shut down some neurons in order to prevent overfitting                                                                                                                                        
    model.add(tf.keras.layers.Dropout(0.5)) # INPUTS NODES TO DROP WITH EACH UPDATE 1 ALL 0 NONE                                                                                                                                 
    # softmax is used as the output layer to provide the confidence of the prediction                                                                                                                                            
    model.add(tf.keras.layers.Dense(noOfClasses,activation='softmax')) # OUTPUT LAYER                                                                                                                                            
    # COMPILE MODEL                                                                                                                                                                                                              
    # Final set up of the model ()                                                                                                                                                                                               
    # RMSprop(): Variant of the gradient descend that uses the RMS error to make an online selection of the learning rate                                                                                                        
    # loss function -> used to make an online estimation of the error, is our optimization correct?                                                                                                                              
    # metrics: used after training to evaluate the model                                                                                                                                                                         
    #     Binary accuracy: % of correct matches                                                                                                                                                                                  
    #     False negatives: % of unlabeled signals                                                                                                                                                                                
    model.compile(optimizer=tf.keras.optimizers.RMSprop(),                                                                                                                                                                       
              loss=tf.keras.losses.BinaryCrossentropy(),                                                                                                                                                                         
              metrics=[tf.keras.metrics.BinaryAccuracy(),                                                                                                                                                                        
                       tf.keras.metrics.FalseNegatives()])                                                                                                                                                                       
                                                                                                                                                                                                                                 
    return model                                                                                                                                                                                                                 



END FUNTION DEF


In [ ]:
################# Parameters #####################
tf.config.set_visible_devices([], 'GPU')

path = "myData" # folder with all the class folders
labelFile = 'labels.csv' # file with all names of classes
batch_size_val=50  # how many to process together before updating the interanl parameters
steps_per_epoch_val=100 # we divide all our database in 10 bathces 
epochs_val=10
imageDimesions = (32,32,3)
testRatio = 0.2    # if 1000 images split will 200 for testing
validationRatio = 0.2 # if 1000 images 20% of remaining 800 will be 160 for validation
###################################################

In [ ]:
############################### Importing of the Images
count = 0
images = []
classNo = []
myList = os.listdir(path)
print("Total Classes Detected:",len(myList))
noOfClasses=len(myList)
print("Importing Classes.....")

#Import names
for x in range (0,len(myList)):
    myPicList = os.listdir(path+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(count)+"/"+y)
        images.append(curImg)
        classNo.append(count)
    # print(count, end =" ")
    count +=1
print(" ")
images = np.array(images)
classNo = np.array(classNo)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
############################### Split Data
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio)
 
# X_train = ARRAY OF IMAGES TO TRAIN
# y_train = CORRESPONDING CLASS ID

In [ ]:
############################### READ CSV FILE
data=pd.read_csv(labelFile)
print("data shape ",data.shape,type(data))

In [ ]:
############################### DISPLAY SOME SAMPLES IMAGES  OF ALL THE CLASSES
num_of_samples = []
cols = 5
num_classes = noOfClasses
fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(5, 300))
fig.tight_layout()
for i in range(cols):
    for j,row in data.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected)- 1), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j)+ "-"+row["Name"])
            num_of_samples.append(len(x_selected))

In [ ]:
############################### DISPLAY A BAR CHART SHOWING NO OF SAMPLES FOR EACH CATEGORY
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")
plt.show()
 

In [ ]:
############################### TRAIN
# Create model structure
model = myModel()
print(model.summary())
# Train the model
history=model.fit_generator(dataGen.flow(X_train,y_train,batch_size=batch_size_val),epochs=epochs_val,validation_data=(X_validation,y_validation),shuffle=1)
model.save('saved_model/my_model')


Results 




In [ ]:
############################### PLOT
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('loss')
plt.xlabel('epoch')
plt.show()
score =model.evaluate(X_test,y_test,verbose=0)
print('Test Score:',score[0])
print('Test Accuracy:',score[1])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2

tf.config.set_visible_devices([], 'GPU')
 
#############################################
 
frameWidth= 640         # CAMERA RESOLUTION
frameHeight = 480
brightness = 180
threshold = 0.75         # PROBABLITY THRESHOLD
font = cv2.FONT_HERSHEY_SIMPLEX
##############################################
 
# SETUP THE VIDEO CAMERA
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, brightness)

model = load_model('saved_model/my_model')
 
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img =cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img
def getCalssName(classNo):
    if   classNo == 0: return 'Speed Limit 20 km/h'
    elif classNo == 1: return 'Speed Limit 30 km/h'
    elif classNo == 2: return 'Speed Limit 50 km/h'
    elif classNo == 3: return 'Speed Limit 60 km/h'
    elif classNo == 4: return 'Speed Limit 70 km/h'
    elif classNo == 5: return 'Speed Limit 80 km/h'
    elif classNo == 6: return 'End of Speed Limit 80 km/h'
    elif classNo == 7: return 'Speed Limit 100 km/h'
    elif classNo == 8: return 'Speed Limit 120 km/h'
    elif classNo == 9: return 'No passing'
    elif classNo == 10: return 'No passing for vechiles over 3.5 metric tons'
    elif classNo == 11: return 'Right-of-way at the next intersection'
    elif classNo == 12: return 'Priority road'
    elif classNo == 13: return 'Yield'
    elif classNo == 14: return 'Stop'
    elif classNo == 15: return 'No vechiles'
    elif classNo == 16: return 'Vechiles over 3.5 metric tons prohibited'
    elif classNo == 17: return 'No entry'
    elif classNo == 18: return 'General caution'
    elif classNo == 19: return 'Dangerous curve to the left'
    elif classNo == 20: return 'Dangerous curve to the right'
    elif classNo == 21: return 'Double curve'
    elif classNo == 22: return 'Bumpy road'
    elif classNo == 23: return 'Slippery road'
    elif classNo == 24: return 'Road narrows on the right'
    elif classNo == 25: return 'Road work'
    elif classNo == 26: return 'Traffic signals'
    elif classNo == 27: return 'Pedestrians'
    elif classNo == 28: return 'Children crossing'
    elif classNo == 29: return 'Bicycles crossing'
    elif classNo == 30: return 'Beware of ice/snow'
    elif classNo == 31: return 'Wild animals crossing'
    elif classNo == 32: return 'End of all speed and passing limits'
    elif classNo == 33: return 'Turn right ahead'
    elif classNo == 34: return 'Turn left ahead'
    elif classNo == 35: return 'Ahead only'
    elif classNo == 36: return 'Go straight or right'
    elif classNo == 37: return 'Go straight or left'
    elif classNo == 38: return 'Keep right'
    elif classNo == 39: return 'Keep left'
    elif classNo == 40: return 'Roundabout mandatory'
    elif classNo == 41: return 'End of no passing'
    elif classNo == 42: return 'End of no passing by vechiles over 3.5 metric tons'
     
while True:
 
    # READ IMAGE
    success, imgOrignal = cap.read()
         
    # PROCESS IMAGE
    img = np.asarray(imgOrignal)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)

    img = img.reshape(1, 32, 32, 1)
    cv2.putText(imgOrignal, "CLASS: " , (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(imgOrignal, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    # PREDICT IMAGE
    predictions = model.predict(img)
    classIndex = model.predict_classes(img)
    probabilityValue =np.amax(predictions)
    if probabilityValue > threshold:
       print(getCalssName(classIndex))
       cv2.putText(imgOrignal,str(classIndex)+" "+str(getCalssName(classIndex)), (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
       cv2.putText(imgOrignal, str(round(probabilityValue*100,2) )+"%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
       cv2.imshow("Result", imgOrignal)
    else:
        print('Not found')
        cv2.putText(imgOrignal, 'No Traffic Sign', (120, 35), font, 0.75,
                    (0, 0, 255), 2, cv2.LINE_AA)

        cv2.imshow("Result", imgOrignal)
         
    if cv2.waitKey(1) and 0xFF == ord('q'):
       break

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('drive/MyDrive/my_model')
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 60)        1560      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 60)        90060     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 60)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 30)        16230     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 30)          8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 30)         0         
 2D)                                                    